In [ ]:
import os,sys
from pathlib import Path


output_dir = Path('xx')
subdirs = ['eval_gpt4v/eval_long_20240518105639', 'eval_d2c/eval_long_20240520083006', 
           'eval_on_llava/eval_long_20240815103907'] # , 'eval_ws/eval_long_20240520031420'
sample_dirs = os.listdir(output_dir / subdirs[0])

In [ ]:
import agents
from html2shot_async import html2shot
from utils import *
from smart_blocker import blocker
from vendors.google import gemini
from vendors.openai import gpt4o
from agents import *
agents.BACKBONE = gpt4o
from PIL import Image
evaluator = Evaluator()

def evaluate(ref, cands):
    listofscore = evaluator.infer([ref] + cands)
    print("")
    print(listofscore)

In [ ]:
from PIL import Image
from IPython.display import display, HTML

def show_comparision(imgs):
    # 
    rows = 5  # 
    image_width = 700  # （）

    #  HTML
    html_content = "<table style='border-collapse: collapse;'>"
    for i in range(0, len(imgs), rows):
        html_content += "<tr>"
        for j in range(rows):
            if i + j < len(imgs):
                img = imgs[i + j]
                # 
                img_resized = img.copy()
                img_resized.thumbnail((image_width, image_width))
                #  HTML  base64 
                from io import BytesIO
                import base64
                buffer = BytesIO()
                img_resized.save(buffer, format="PNG")
                img_base64 = base64.b64encode(buffer.getvalue()).decode()
                html_content += f"<td style='padding: 10px;'><img src='data:image/png;base64,{img_base64}' /></td>"
        html_content += "</tr>"
    html_content += "</table>"

    #  Notebook 
    display(HTML(html_content))


In [ ]:
import random 
# id = random.randint(0, len(sample_dirs)-1)
id=3
samples = [output_dir / f'{s}/{sample_dirs[id]}' for s in subdirs if (output_dir / f'{s}/{sample_dirs[id]}').exists()]
ref = Image.open( samples[0] / 'answer.png' )
cands = [Image.open(s / 'prediction.png') for s in samples]
evaluate(ref, cands)
show_comparision([ref] + cands)

In [ ]:
output_dir_1 = Path('xx')
ref=Image.open(output_dir_1/'image.png')
cands = [Image.open(output_dir_1/'prediction_101.png'),Image.open(output_dir_1/'prediction_103.png'),Image.open(output_dir_1/'prediction_1000.png')]
evaluate(ref, cands)
show_comparision([ref] + cands)

改硬性指标(上面的没用了)

In [ ]:
import sys
from pathlib import Path
from PIL import Image
import json
import os
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
# 
sys.path.append("xx")

# 
from evaluation.mrweb.emd_similarity import emd_similarity
from evaluation.mrweb.study import CLIPScorer, LPIPSScorer, ssim_score, psnr_score, mae_score


def evaluate_images(img1, img2):
    """
    ，。
    """
    try:
        clip_scorer = CLIPScorer(model_name='ViT-B-32-quickgelu', pretrained='openai')
        lpips_scorer = LPIPSScorer(net='vgg')

        clip_score = clip_scorer.score(img1, img2)
        lpips_score = lpips_scorer.score(img1, img2)
        ssim_value = ssim_score(img1, img2)
        psnr_value = psnr_score(img1, img2)
        mae_value = mae_score(img1, img2)
        emd_score = emd_similarity(img1, img2, max_size=96, mode="RGB")

        return {
            "CLIP Score": clip_score,
            "LPIPS Score": lpips_score,
            "SSIM": ssim_value,
            "PSNR": psnr_value,
            "MAE": mae_value,
            "EMD Similarity": emd_score
        }
    except Exception as e:
        print(f"Error during evaluation: {e}")
        return None


def evaluate_all(ref_image, candidate_images):
    """
    ，。
    """
    results = []
    for idx, cand_image in enumerate(candidate_images):
        try:
            print(f"Evaluating candidate {idx + 1}/{len(candidate_images)}...")
            scores = evaluate_images(ref_image, cand_image)
            if scores:
                results.append({
                    "candidate_index": idx,
                    "scores": scores
                })
                # 
                print(f"Results for candidate {idx}:\n" + "\n".join(
                    [f"{metric}: {score}" for metric, score in scores.items()]))
            else:
                print(f"Failed to evaluate candidate {idx}")
        except Exception as e:
            print(f"Error evaluating candidate {idx}: {e}")

    
    return results



In [ ]:
if __name__ == "__main__":
    # 
    output_dir_1 = Path('xx')
    try:
        ref = Image.open(output_dir_1 / 'image.png')
        cands = [Image.open(output_dir_1/'prediction_0.png'),Image.open(output_dir_1/'prediction_1000.png'),Image.open(output_dir_1/'prediction_100000.png')]
        # 
        results = evaluate_all(ref, cands)
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    show_comparision([ref] + cands)


In [ ]:
if __name__ == "__main__":
    # 
    output_dir_1 = Path('xx')
    try:
        ref=Image.open(output_dir_1/'image.png')
        cands = [Image.open(output_dir_1/'prediction_0.png'),Image.open(output_dir_1/'prediction_1000.png'),Image.open(output_dir_1/'prediction_1001.png')]
        results = evaluate_all(ref, cands)
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    show_comparision([ref] + cands)



In [ ]:
# wgh
if __name__ == "__main__":
    # 
    output_dir_1 = Path('xx')
    try:
        ref=Image.open(output_dir_1/'answer.png')
        cands = [Image.open(output_dir_1/'assemble_0.png'),Image.open(output_dir_1/'assemble_1.png'),Image.open(output_dir_1/'assemble_2.png'),Image.open(output_dir_1/'assemble_3.png')]
        results = evaluate_all(ref, cands)
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    show_comparision([ref] + cands)


In [ ]:
if __name__ == "__main__":
    # 
    output_dir_1 = Path('xx')
    try:
        ref=Image.open(output_dir_1/'answer.png')
        cands = [Image.open(output_dir_1/'assemble_0.png'),Image.open(output_dir_1/'assemble_1.png'),Image.open(output_dir_1/'assemble_2.png'),Image.open(output_dir_1/'assemble_3.png')]
        results = evaluate_all(ref, cands)
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    show_comparision([ref] + cands)


In [ ]:
# ，
from tqdm import tqdm
from agents import *
from smart_blocker import *
save_path=f'xx'
base_path = save_path
agentGenerate = AgentGenerate()

def generate_module_code(plans):
    md5 = image2md5(image)
    codes_plans = []
    for index,plan in tqdm(enumerate(plans,start=1),total=len(plans)):
        module_image = crop_image(image,plan)
        path = save_path + f"/{md5}"
        os.makedirs(path, exist_ok=True)
        path += f"/{index}.png"

        print(path)
        module_image.save(path)

        # code = agentGenerate.infer(image1=image,image2=module_image,text=json.dumps(plan))
        code = agentGenerate.infer([module_image])
        code=remove_code_markers(code)
        print(code)

        image_pre = html2shot(base_path, code,md5,index)

        codes_plans.append({"module_position":plan,"module_code":code})
    
    return codes_plans
image = Image.open("xx")
# 
plans = blocker(image)
codes = generate_module_code(plans)

In [ ]:
codes

In [ ]:
def image2md5(image):
    image_bytes = io.BytesIO()
    image.save(image_bytes, format='PNG')
    image_data = image_bytes.getvalue()
    md5_hash = hashlib.md5(image_data)
    md5_hex = md5_hash.hexdigest()
    return str(md5_hex)
# 
def absolute_assemble(image,code_plans):
    md5 = image2md5(image)

    html_image = []

    code_abss = []
    for node in code_plans:
        bbox = node['module_position']
        code_abs = f'<div style="position: absolute; overflow: hidden; border: 1px solid blue; left: {round(bbox[0] * image.width)}px; top: {round(bbox[1] * image.height)}px; width: {round((bbox[2] - bbox[0]) * image.width)}px; height: {round((bbox[3] - bbox[1]) * image.height)}px;">{"".join(node["module_code"])}</div>'
        code_abss.append(code_abs)

    html = """<html>
    <body>
        """ + '\n'.join(code_abss) + """
    </body>
</html>"""

    # imageofhtml = html2shot(base_path, html_content=html,md5=md5,index=1000)
    # html_image.append({"html":html,"image":imageofhtml})
    return html


html1code = absolute_assemble(image,codes)

md5 = image2md5(image=image)
print(md5)
imagehtml = await html2shot(save_path,html1code,md5,22)

In [ ]:
# agent
agentAssemble = AgentAssemble()

html2code = agentAssemble.infer([image],text=json.dumps(codes))
imagehtml2 = await html2shot(save_path,html2code,md5,33)

In [ ]:
if __name__ == "__main__":
    # 
    output_dir_1 = Path('xx')

    try:
        ref=Image.open(output_dir_1/'image.png')
        cands = [Image.open(output_dir_1/'prediction_0.png'),Image.open(output_dir_1/'prediction_1000.png'),Image.open(output_dir_1/'prediction_100000.png')]
        results = evaluate_all(ref, cands)
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    show_comparision([ref] + cands)

In [ ]:
print(type(results))

In [ ]:
results

In [ ]:
# 
def normalize_value(value, min_value, max_value, reverse=False):
    normalized_value = (value - min_value) / (max_value - min_value)
    if reverse:
        # （）， 1 - normalized_value
        return 1 - normalized_value
    return normalized_value

def get_best_candidate(result, mae_emd_threshold=0.1):
    best_index = -1

    # MAEEMD，
    all_mae = [entry['scores']['MAE'] for entry in result]
    all_emd = [entry['scores']['EMD Similarity'] for entry in result]
    
    min_mae = min(all_mae)
    max_mae = max(all_mae)
    min_emd = min(all_emd)
    max_emd = max(all_emd)

    scores_with_index = []

    for i, entry in enumerate(result):
        scores = entry['scores']
        mae = scores['MAE']
        emd = scores['EMD Similarity']
        clip_score = scores['CLIP Score']

        # MAE（）EMD
        normalized_mae = normalize_value(mae, min_mae, max_mae, reverse=True)
        normalized_emd = normalize_value(emd, min_emd, max_emd, reverse=False)

        # 
        print(f"index = {i}")
        print(f"mae : {normalized_mae}")
        print(f"emd : {normalized_emd}")

        # MAEEMD（50%）
        combined_score = 0.5 * normalized_mae + 0.5 * normalized_emd

        # 
        scores_with_index.append((i, combined_score, mae, emd, clip_score))

    # 
    print(scores_with_index)

    # ，
    scores_with_index.sort(key=lambda x: x[1] , reverse= True)

    # 
    best_index, best_combined_score, _, _, _ = scores_with_index[0]
    second_best_index, second_best_combined_score, _, _, _ = scores_with_index[1]

    # ，
    score_diff = best_combined_score - second_best_combined_score
    print(f"score_diff {score_diff}")

    print(f"Best candidate: {best_index}, combined score: {best_combined_score}")
    print(f"Second best candidate: {second_best_index}, combined score: {second_best_combined_score}")

    if score_diff > mae_emd_threshold:  # 
        print(f"Candidate {best_index} is significantly better, selecting as the best.")
        return best_index
    else:
        # ， CLIP Score

        #  0.1 
        valid_candidates = []
        for index, combined_score, _, _, clip_score in scores_with_index:
            if abs(combined_score - best_combined_score) <= 0.1:
                valid_candidates.append({"index":index,"clip_score":clip_score})
        if valid_candidates:
            best_candidate = max(valid_candidates, key=lambda x: x["clip_score"])
            best_clip_index = best_candidate["index"]
            print(f"clip {best_clip_index}")
            return best_clip_index
        else:
            # ，best_index
            print("")
            return best_index

In [ ]:
results

In [ ]:
results_index = get_best_candidate(result=results)
results_index

In [ ]:
from evaluation.mrweb.emd_similarity import emd_similarity
from evaluation.mrweb.study import CLIPScorer, LPIPSScorer, ssim_score, psnr_score, mae_score
from evaluation.metrics import clip_sim

def evaluate_images(img1, img2):
    """
    ，。
    """
    try:
        #  MAE
        mae_value = mae_score(img1, img2)
        #  EMD
        emd_score = emd_similarity(img1, img2, max_size=96, mode="RGB") 
        #  CLIP
        clip_value = clip_sim(img1, img2)  

        return mae_value, emd_score, clip_value
    except Exception as e:
        logger.info(f"Error during evaluation: {e}")
        return None



def verify_score(mae, nemd, clip_similarity,  weights=(0.3, 0.3, 0.4)):
    """
    Computes a composite similarity score based on MAE, NEMD, and CLIP similarity.

    Args:
        mae (float): Mean Absolute Error between images, normalized.
        nemd (float): 1-emd, and Normalized Earth Mover's Distance (already in [0, 1]).
        clip_similarity (float): CLIP cosine similarity between images.
        weights (tuple): Weights for (MAE_similarity, NEMD_similarity, CLIP_transformed).

    Returns:
        float: Composite similarity score.
    """
    w1, w2, w3 = weights    
    
    # Compute composite score
    composite_score = w1 * 1 / (1 + mae) + w2 * nemd + w3 * clip_similarity ** 0.5
    
    return composite_score
     

def get_best_candidate(ref_img, cand_imgs):
    assert len(cand_imgs) > 1, "There must be more than one candidates to select."
    
    scores = {'MAE':[], 'NEMD':[], 'CLIP':[]}
    for i,cand_image in  enumerate(cand_imgs):
        mae_value, emd_score, clip_value = evaluate_images(ref_img, cand_image)

        print(f"Candidate {i+1} MAE score: {mae_value}")
        print(f"Candidate {i+1} EMD score: {emd_score}")
        print(f"Candidate {i+1} CLIP score: {clip_value}")
        scores['MAE'].append(mae_value)
        scores['NEMD'].append(emd_score)
        scores['CLIP'].append(clip_value)
    print(scores)
    # 
    weights = (0.3, 0.3, 0.4)
    # 
    final_scores = [verify_score(scores['MAE'][i], scores['NEMD'][i], scores['CLIP'][i], weights) for i in range(len(cand_imgs))]
    print(final_scores)
    
    # 
    max_score = max(final_scores)
    idx = final_scores.index(max_score)
    
    return idx


In [ ]:
from evaluation.mrweb.emd_similarity import emd_similarity
from evaluation.mrweb.study import CLIPScorer, LPIPSScorer, ssim_score, psnr_score, mae_score
from evaluation.metrics import clip_sim
import numpy as np
from PIL import Image
from IPython.display import display, HTML
import os,sys
from pathlib import Path

def show_comparision(imgs):
    # 
    rows = 5  # 
    image_width = 700  # （）

    #  HTML
    html_content = "<table style='border-collapse: collapse;'>"
    for i in range(0, len(imgs), rows):
        html_content += "<tr>"
        for j in range(rows):
            if i + j < len(imgs):
                img = imgs[i + j]
                # 
                img_resized = img.copy()
                img_resized.thumbnail((image_width, image_width))
                #  HTML  base64 
                from io import BytesIO
                import base64
                buffer = BytesIO()
                img_resized.save(buffer, format="PNG")
                img_base64 = base64.b64encode(buffer.getvalue()).decode()
                html_content += f"<td style='padding: 10px;'><img src='data:image/png;base64,{img_base64}' /></td>"
        html_content += "</tr>"
    html_content += "</table>"

    #  Notebook 
    display(HTML(html_content))

def normalize_value(value, min_value, max_value, reverse=False):
    normalized_value = (value - min_value) / (max_value - min_value)
    if reverse:
        return 1 - normalized_value
    return normalized_value

def evaluate_images(img1, img2):
    """
    ，。
    """
    try:
        mae_value = mae_score(img1, img2)
        emd_score = emd_similarity(img1, img2, max_size=96, mode="RGB") 
        clip_value = clip_sim(img1, img2)  
        return mae_value, emd_score, clip_value
    except Exception as e:
        logger.info(f"Error during evaluation: {e}")
        return None
    
# 
def verify_score(mae, nemd, clip_similarity, weights=(0.3, 0.3, 0.4)):
    w1, w2, w3 = weights
    composite_score = w1 * 1 / (1 + mae) + w2 * nemd + w3 * clip_similarity ** 0.5
    return composite_score

def get_best_candidate(ref_img, cand_imgs):
    assert len(cand_imgs) > 1, "There must be more than one candidates to select."
    
    scores = {'MAE':[], 'NEMD':[], 'CLIP':[]}
    mae_values, emd_scores, clip_values = [], [], []
    
    # 
    for i, cand_image in enumerate(cand_imgs):
        mae_value, emd_score, clip_value = evaluate_images(ref_img, cand_image)        
        mae_values.append(mae_value)
        emd_scores.append(emd_score)
        clip_values.append(clip_value)
    
    # 
    mae_min, mae_max = np.min(mae_values), np.max(mae_values)
    emd_min, emd_max = np.min(emd_scores), np.max(emd_scores)
    clip_min, clip_max = np.min(clip_values), np.max(clip_values)
    
    # 
    normalized_mae = [normalize_value(val, mae_min, mae_max,reverse=True) for val in mae_values]
    normalized_nemd = [normalize_value(val, emd_min, emd_max) for val in emd_scores]
    normalized_clip = [normalize_value(val, clip_min, clip_max) for val in clip_values]
    
    for i in range(len(mae_values)):
        print(f"Candidate {i+1}:")
        print(f"  Original MAE: {mae_values[i]}, Normalized MAE: {normalized_mae[i]}")
        print(f"  Original NEMD: {emd_scores[i]}, Normalized NEMD: {normalized_nemd[i]}")
        print(f"  Original CLIP: {clip_values[i]}, Normalized CLIP: {normalized_clip[i]}")

    
    # 
    selected_indices = []
    for i in range(len(cand_imgs)):
        high_scores = sum([1 for score in [normalized_mae[i], normalized_nemd[i], normalized_clip[i]] if score > 0.9])
        if high_scores >= 2:
            selected_indices.append(i)
    
    # ，
    if len(selected_indices) == 1:
        best_index = selected_indices[0]
    elif len(selected_indices) > 1:
        # ，
        composite_scores = []
        for i in selected_indices:
            composite_score = verify_score(mae_values[i], emd_scores[i],clip_values[i])
            composite_scores.append(composite_score)
        best_index = selected_indices[np.argmax(composite_scores)]
    else:
        # ，
        composite_scores = []
        for i in range(len(cand_imgs)):
            composite_score = verify_score(mae_values[i], emd_scores[i],clip_values[i])
            composite_scores.append(composite_score)
        best_index = np.argmax(composite_scores)
    
    print(f"Best candidate index: {best_index}")
    return best_index

In [ ]:
from evaluation.mrweb.emd_similarity import emd_similarity
from evaluation.mrweb.study import CLIPScorer, LPIPSScorer, ssim_score, psnr_score, mae_score
from evaluation.metrics import clip_sim
import numpy as np
from PIL import Image
from IPython.display import display, HTML
import os,sys
from pathlib import Path

def show_comparision(imgs):
    # 
    rows = 5  # 
    image_width = 700  # （）

    #  HTML
    html_content = "<table style='border-collapse: collapse;'>"
    for i in range(0, len(imgs), rows):
        html_content += "<tr>"
        for j in range(rows):
            if i + j < len(imgs):
                img = imgs[i + j]
                # 
                img_resized = img.copy()
                img_resized.thumbnail((image_width, image_width))
                #  HTML  base64 
                from io import BytesIO
                import base64
                buffer = BytesIO()
                img_resized.save(buffer, format="PNG")
                img_base64 = base64.b64encode(buffer.getvalue()).decode()
                html_content += f"<td style='padding: 10px;'><img src='data:image/png;base64,{img_base64}' /></td>"
        html_content += "</tr>"
    html_content += "</table>"

    #  Notebook 
    display(HTML(html_content))

def normalize_value(value, min_value, max_value, reverse=False):
    normalized_value = (value - min_value) / (max_value - min_value)
    if reverse:
        return 1 - normalized_value
    return normalized_value

def evaluate_images(img1, img2):
    """
    ，。
    """
    try:
        mae_value = mae_score(img1, img2)
        emd_score = emd_similarity(img1, img2, max_size=96, mode="RGB") 
        clip_value = clip_sim(img1, img2)  
        return mae_value, emd_score, clip_value
    except Exception as e:
        logger.info(f"Error during evaluation: {e}")
        return None
    
# 
def verify_score(mae, nemd, clip_similarity, weights=(0.3, 0.3, 0.4)):
    w1, w2, w3 = weights
    composite_score = w1 * (1-mae/100) + w2 * nemd + w3 * clip_similarity ** 0.5
    return composite_score

def get_best_candidate(ref_img, cand_imgs):
    assert len(cand_imgs) > 1, "There must be more than one candidates to select."
    
    # 
    mae_values, emd_scores, clip_values = [], [], []
    for cand_image in cand_imgs:
        mae_value, emd_score, clip_value = evaluate_images(ref_img, cand_image)
        mae_values.append(mae_value)
        emd_scores.append(emd_score)
        clip_values.append(clip_value)
    
    # 
    mae_threshold = 0.1 * np.min(mae_values)  # MAE10%
    emd_threshold = 0.1  # EMD0.1
    clip_threshold = 0.1  # CLIP0.1
    
    # 
    scores = [0] * len(cand_imgs)
    
    # MAE
    if np.max(mae_values) - np.min(mae_values) > mae_threshold:
        for i, mae in enumerate(mae_values):
            # MAE：(max_mae - mae) / (max_mae - min_mae)
            mae_score = (np.max(mae_values) - mae) / (np.max(mae_values) - np.min(mae_values))
            scores[i] += mae_score

    # EMD
    if np.max(emd_scores) - np.min(emd_scores) > emd_threshold:
        for i, emd in enumerate(emd_scores):
            # EMD：(emd - min_emd) / (max_emd - min_emd)
            emd_score = (emd - np.min(emd_scores)) / (np.max(emd_scores) - np.min(emd_scores))
            scores[i] += emd_score

    # CLIP
    if np.max(clip_values) - np.min(clip_values) > clip_threshold:
        for i, clip in enumerate(clip_values):
            # CLIP：(clip - min_clip) / (max_clip - min_clip)
            clip_score = (clip - np.min(clip_values)) / (np.max(clip_values) - np.min(clip_values))
            scores[i] += clip_score
    
    # 
    for i in range(len(cand_imgs)):
        print(f"Candidate {i}:")
        print(f"  MAE: {mae_values[i]}, EMD: {emd_scores[i]}, CLIP: {clip_values[i]}")
        print(f"  Score: {scores[i]}")
    
    # 0，
    if all(score == 0 for score in scores):
        composite_scores = []
        for i in range(len(cand_imgs)):
            composite_score = verify_score(mae_values[i], emd_scores[i], clip_values[i])
            composite_scores.append(composite_score)
        best_index = np.argmax(composite_scores)
    else:
        # 
        best_index = np.argmax(scores)
    
    print(f"Best candidate index: {best_index}")
    return best_index